In [1369]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd

In [1370]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

In [1371]:
init_notebook_mode(connected=True)

In [1372]:
#Load csv file of senate election data from 1976-2018
df = pd.read_csv('1976-2018-senate.csv', encoding = 'unicode_escape')

In [1373]:
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,...,candidate_scale,Vote_Percent_abs,Party_Label,text,Points,Year_Abrv,https://ballotpedia.org/,Name_Underscore,site,link
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,-1,0.433394,R,Sam Steiger (R) 43.3%<br>,43.3%,76,https://ballotpedia.org/Sam_Steiger,Sam_Steiger,"""https://ballotpedia.org/Sam_Steiger""","<a href=""https://ballotpedia.org/Sam_Steiger"">..."
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,1,0.002111,I,Wm. Mathews Feighan (I) 0.2%<br>,0.2%,76,https://ballotpedia.org/Wm._Mathews_Feighan,Wm._Mathews_Feighan,"""https://ballotpedia.org/Wm._Mathews_Feighan""","<a href=""https://ballotpedia.org/Wm._Mathews_F..."
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,1,0.540109,D,Dennis DeConcini (D) 54%<br>,54%,76,https://ballotpedia.org/Dennis_DeConcini,Dennis_DeConcini,"""https://ballotpedia.org/Dennis_DeConcini""","<a href=""https://ballotpedia.org/Dennis_DeConc..."
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,1,0.009862,L,Allan Norwitz (L) 1%<br>,1%,76,https://ballotpedia.org/Allan_Norwitz,Allan_Norwitz,"""https://ballotpedia.org/Allan_Norwitz""","<a href=""https://ballotpedia.org/Allan_Norwitz..."
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,1,0.014524,I,Bob Field (I) 1.5%<br>,1.5%,76,https://ballotpedia.org/Bob_Field,Bob_Field,"""https://ballotpedia.org/Bob_Field""","<a href=""https://ballotpedia.org/Bob_Field""> B..."


In [1414]:
#Create year slider functionality
data_slider = []

#Group Each election by year and state
for year in df['year'].unique():    
    y =  df[(df['year']== year)]  
    y = y[(y['special'] == False )]
    y = y[(y['stage'] == 'gen' )]
    y = y.sort_values('Vote_Percent_abs')
    df_segmented = y.loc[y.groupby('state_po')['Vote_Percent_abs'].idxmax()]
#     df_segmented.sort_values('state', ascending=False)

    y.loc[y.groupby('state_po')['Vote_Percent_abs'].idxmax()]=0
  
            
#Create hover text for second place in race   
    df_segmented2 = y.loc[y.groupby('state_po')['Vote_Percent_abs'].idxmax()]
    df_segmented2 = df_segmented2.drop(df_segmented2[df_segmented2.year == 0].index)
#     df_segmented2.sort_values('state', ascending=False)


    if df_segmented['Vote_Percent'].max() == df_segmented['Vote_Percent_abs'].max(): 
            filler_highest_value = df_segmented['Vote_Percent'].max()
            filler_lowest_value = df_segmented['Vote_Percent'].max()*(-1)

    else:
        if df_segmented['Vote_Percent_abs'].max()<.9:
            filler_highest_value = df_segmented['Vote_Percent_abs'].max()
            filler_lowest_value = df_segmented['Vote_Percent_abs'].max()*(-1)
            
        else: 
            filler_highest_value = .76
            filler_lowest_value = -.76
            
    
    
    #Make different color in map for independents who win    
    for i in range(len(df_segmented)):
        if df_segmented['party'].values[i].startswith('independent'):
            df_segmented['Vote_Percent'].values[i] = filler_highest_value+.001
    
    
    for i in range(len(df_segmented2)):
        if df_segmented['Vote_Percent_abs'].values[i] < 1 and df_segmented['state'].values[i] == df_segmented2['state'].values[i]:
            df_segmented['mode'].values[i] = df_segmented2['text'].values[i]
        
        else:
            df_segmented['mode'].values[i] = ''

#     print(len(df_segmented['mode']))
#     print(len(df_segmented2['text']))

#Create Choropleth map of United States        
    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['state_po'],
                        hoverinfo="text",
                        z=df_segmented['Vote_Percent'],
                        locationmode='USA-states',
#                         text = '<a href="http://www.google.com">Link Link Link Link</a>',
                        text = df_segmented['text'] + df_segmented['mode'],
                        marker = dict(line = dict(color = 'rgb(12,12,12)', width=1.5)),
                        zauto=False,
                        zmax = filler_highest_value+.001,
                        zmin = filler_lowest_value,
                        colorscale = [[0, '#8b0000'], 
               [0.05, '#c7152d'],
               [0.1, '#e60000'],
               [0.15, '#ffcccb'],
               [0.5, '#FFFFFF'],
               [0.85, '#99bbff'],
               [0.9, '#4d88ff'],                       
               [0.95, '#0000e6'],         
               [0.9999, '#00008B'],
               [1, '#008000']])


    data_slider.append(data_each_yr)

In [1415]:
#Put slider on map
steps = []
for i in range(len(data_slider)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(data_slider)}, # layout attribute
              {"title":str(i*2 + 1976) + ' Senate Map'}],
        label='{}'.format(i*2 + 1976)
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

In [1425]:
sliders = [dict(active=21, pad={"t": 1}, steps=steps)]

title = 'Slide the Scale to View Senate Election Results for a Specific Year'

layout = dict(title = title, geo=dict(scope='usa', showlakes = True,lakecolor='rgb(85,173,240)'
            ),
              sliders=sliders)

In [1427]:
choromap = go.Figure(data = data_slider,layout = layout)

choromap.update_traces(showscale=False)

plot(choromap)

'temp-plot.html'